# Converting Cartesian ADM Initial Data to BSSN Curvilinear Initial Data, Using Brill-Lindquist Initial Data as an Example

### NRPy+ Source Code for this module: [BSSN/CartesianADMID_to_BSSNCurvilinearID.py](../edit/BSSN/CartesianADMID_to_BSSNCurvilinearID.py)

Here we use NRPy+ to first convert the ADM variables

$$\left\{\gamma_{ij}, K_{ij}, \alpha, \beta^i\right\}$$

to the BSSN variables

$$\left\{\bar{\gamma}_{i j},\bar{A}_{i j},\phi, K, \bar{\Lambda}^{i}, \alpha, \beta^i, B^i\right\},$$ 

and the BSSN variables to the rescaled BSSNCurvilinear variables (as defined in [the BSSN Curvilinear tutorial](Tutorial-BSSNCurvilinear.ipynb)):

$$\left\{h_{i j},a_{i j},\phi, K, \lambda^{i}, \alpha, \mathcal{V}^i, \mathcal{B}^i\right\}.$$ 


First we have (Eqs. 2 and 3 of [Ruchlin *et al.*](https://arxiv.org/pdf/1712.07658.pdf)):
$$
\bar{\gamma}_{i j} = \left(\frac{\bar{\gamma}}{\gamma}\right)^{1/3} \gamma_{ij}.
$$

Next we apply two important facts:

1. We choose $\bar{\gamma}=\hat{\gamma}$. 
1. In Cartesian coordinates, $\hat{\gamma}=1$. 

Therefore, for initial data in Cartesian coordinates, we have
$$
\bar{\gamma}_{i j} = \gamma^{-1/3} \gamma_{ij}.
$$

In [1]:
# Step P1: Import needed modules
import sympy as sp
import NRPy_param_funcs as par
from outputC import *
import indexedexp as ixp
import reference_metric as rfm
import BSSN.BSSN_RHSs as bssn # needed for parameters

# Step P2: Import Brill-Lindquist initial data
import BSSN.BrillLindquist as bl
bl.BrillLindquist()
Cartxyz = bl.Cartxyz
gammaCartDD = bl.gammaCartDD
KCartDD = bl.KCartDD
alphaCart = bl.alphaCart
betaCartU = bl.betaCartU
BCartU = bl.BCartU

# The ADM & BSSN formalisms only work in 3D; they are 3+1 decompositions of Einstein's equations.
#    To implement axisymmetry or spherical symmetry, simply set all spatial derivatives in
#    the relevant angular directions to zero; DO NOT SET DIM TO ANYTHING BUT 3.
# Step 0: Set spatial dimension (must be 3 for BSSN)
DIM = 3

# Step 1: Convert ADM $\gamma_{ij}$ to BSSN $\bar{\gamma}_{ij}$
gammaCartUU, gammaCartDET = ixp.symm_matrix_inverter3x3(gammaCartDD)
gammaCartbarDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        gammaCartbarDD[i][j] = gammaCartDET**(-sp.Rational(1,3))*gammaCartDD[i][j]

Second, we convert the extrinsic curvature $K_{ij}$ to the trace-free extrinsic curvature $\bar{A}_{ij}$, plus the trace of the extrinsic curvature $K$, where (Eq. 3 of [Baumgarte *et al.*](https://arxiv.org/pdf/1211.6632.pdf)):

\begin{align}
K &= \gamma^{ij} K_{ij} \\
\bar{A}_{ij} &= \left(\frac{\bar{\gamma}}{\gamma}\right)^{1/3} \left(K_{ij} - \frac{1}{3} \gamma_{ij} K \right)\\
&= \gamma^{-1/3} \left(K_{ij} - \frac{1}{3} \gamma_{ij} K \right)
\end{align}

In [2]:
# Step 2: Convert ADM $K_{ij}$ to $\bar{A}_{ij}$ and $K$, 
#          where (Eq. 3 of Baumgarte et al.: https://arxiv.org/pdf/1211.6632.pdf)
trKCart = sp.sympify(0)
for i in range(DIM):
    for j in range(DIM):
        trKCart += gammaCartUU[i][j]*KCartDD[i][j]

ACartbarDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        ACartbarDD[i][j] = gammaCartDET**(-sp.Rational(1,3))*\
          (KCartDD[i][j] - sp.Rational(1,3)*gammaCartDD[i][j]*trKCart)

Third, we define $\bar{\Lambda}^i$ (Eqs. 4 and 5 of [Baumgarte *et al.*](https://arxiv.org/pdf/1211.6632.pdf)):

$$
\bar{\Lambda}^i = \bar{\gamma}^{jk}\left(\bar{\Gamma}^i_{jk} - \hat{\Gamma}^i_{jk}\right).
$$

In Cartesian coordinates, $\hat{\Gamma}^i_{jk}=0$, so we get in Cartesian coordinates
\begin{align}
\bar{\Lambda}^i &= \bar{\gamma}^{jk} \bar{\Gamma}^i_{jk} \\
&= \bar{\gamma}^{jk} \frac{1}{2} \bar{\gamma}^{il} \left(
\bar{\gamma}_{lj,k} + \bar{\gamma}_{lk,j} - \bar{\gamma}_{jk,l} \right)
\end{align}

In [3]:
# Step 3: Define $\bar{\Lambda}^i$ from Eqs. 4 and 5 of Baumgarte et al.: https://arxiv.org/pdf/1211.6632.pdf
LambdaCartbarU = ixp.zerorank1()
# Need to compute \bar{\gamma}^{ij} from \bar{\gamma}_{ij}:
gammaCartbarUU, gammaCartbarDET = ixp.symm_matrix_inverter3x3(gammaCartbarDD)

for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            for l in range(DIM):
                LambdaCartbarU[i] += sp.Rational(1,2)*gammaCartbarUU[j][k]*gammaCartbarUU[i][l]* \
                                        ( sp.diff(gammaCartbarUU[l][j],Cartxyz[k]) + 
                                          sp.diff(gammaCartbarUU[l][k],Cartxyz[j]) -
                                          sp.diff(gammaCartbarUU[j][k],Cartxyz[l]) )

All BSSN tensors and vectors are in Cartesian coordinates $x^i_{\rm Cart} = (x,y,z)$, but we need them in the curvilinear coordinate system $x^i_{\rm rfm}= ({\rm xx0},{\rm xx1},{\rm xx2})$ set by the "reference_metric::CoordSystem" variable. Empirically speaking, it is far easier to write $(x({\rm xx0},{\rm xx1},{\rm xx2}),y({\rm xx0},{\rm xx1},{\rm xx2}),z({\rm xx0},{\rm xx1},{\rm xx2}))$ than the inverse, so we will compute the Jacobian matrix

$$
{\rm Jac\_dUCart\_dDrfmUD[i][j]} = \frac{\partial x^i_{\rm Cart}}{\partial x^j_{\rm rfm}},
$$

via exact differentiation (courtesy SymPy), and the inverse Jacobian
$$
{\rm Jac\_dUrfm\_dDCartUD[i][j]} = \frac{\partial x^i_{\rm rfm}}{\partial x^j_{\rm Cart}},
$$

using NRPy+'s ${\rm generic\_matrix\_inverter3x3()}$ function. In terms of these, the transformation of BSSN tensors from Cartesian to "reference_metric::CoordSystem" coordinates may be written:

\begin{align}
\bar{\gamma}^{\rm rfm}_{ij} &= 
\frac{\partial x^\ell_{\rm Cart}}{\partial x^i_{\rm rfm}}
\frac{\partial x^m_{\rm Cart}}{\partial x^j_{\rm rfm}} \bar{\gamma}^{\rm Cart}_{\ell m}\\
\bar{A}^{\rm rfm}_{ij} &= 
\frac{\partial x^\ell_{\rm Cart}}{\partial x^i_{\rm rfm}}
\frac{\partial x^m_{\rm Cart}}{\partial x^j_{\rm rfm}} \bar{A}^{\rm Cart}_{\ell m}\\
\bar{\Lambda}^i_{\rm rfm} &= \frac{\partial x^i_{\rm rfm}}{\partial x^\ell_{\rm Cart}} \bar{\Lambda}^\ell_{\rm Cart} \\
\beta^i_{\rm rfm} &= \frac{\partial x^i_{\rm rfm}}{\partial x^\ell_{\rm Cart}} \beta^\ell_{\rm Cart}
\end{align}

In [4]:
# Step 4: Convert BSSN tensors to basis specified by CoordSystem variable above.

# trK and alpha are scalars, so no Jacobian transformations are necessary.
trK   = trKCart
alpha = alphaCart

Jac_dUCart_dDrfmUD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        Jac_dUCart_dDrfmUD[i][j] = sp.diff(rfm.xxCart[i],rfm.xx[j])

Jac_dUrfm_dDCartUD, dummyDET = ixp.generic_matrix_inverter3x3(Jac_dUCart_dDrfmUD)

gammaDD    = ixp.zerorank2()
gammabarDD = ixp.zerorank2()
AbarDD     = ixp.zerorank2()
LambdabarU = ixp.zerorank1()
betaU      = ixp.zerorank1()
BU         = ixp.zerorank1()
for i in range(DIM):
    for j in range(DIM):
        LambdabarU[i] += Jac_dUrfm_dDCartUD[i][j]*LambdaCartbarU[j]
        betaU[i]      += Jac_dUrfm_dDCartUD[i][j]*betaCartU[j]
        BU[i]         += Jac_dUrfm_dDCartUD[i][j]*BCartU[j]
        for k in range(DIM):
            for l in range(DIM):
                gammaDD[i][j]    += Jac_dUCart_dDrfmUD[k][i]*Jac_dUCart_dDrfmUD[l][j]*   gammaCartDD[k][l]
                gammabarDD[i][j] += Jac_dUCart_dDrfmUD[k][i]*Jac_dUCart_dDrfmUD[l][j]*gammaCartbarDD[k][l]
                AbarDD[i][j]     += Jac_dUCart_dDrfmUD[k][i]*Jac_dUCart_dDrfmUD[l][j]*    ACartbarDD[k][l]

Next we set the conformal factor variable $\texttt{cf}$, which is set by the "BSSN_RHSs::ConformalFactor" parameter. For example if "ConformalFactor" is set to "phi", we can use Eq. 3 of [Ruchlin *et al.*](https://arxiv.org/pdf/1712.07658.pdf), which in arbitrary coordinates is written:

$$
\phi = \frac{1}{12} \log\left(\frac{\gamma}{\bar{\gamma}}\right).
$$

Alternatively if "BSSN_RHSs::ConformalFactor" is set to "chi", then
$$
\chi = e^{-4 \phi} = \exp\left(-4 \frac{1}{12} \left(\frac{\gamma}{\bar{\gamma}}\right)\right) 
= \exp\left(-\frac{1}{3} \log\left(\frac{\gamma}{\bar{\gamma}}\right)\right) = \left(\frac{\gamma}{\bar{\gamma}}\right)^{-1/3}.
$$

Finally if "BSSN_RHSs::ConformalFactor" is set to "W", then
$$
W = e^{-2 \phi} = \exp\left(-2 \frac{1}{12} \log\left(\frac{\gamma}{\bar{\gamma}}\right)\right) = 
\exp\left(-\frac{1}{6} \log\left(\frac{\gamma}{\bar{\gamma}}\right)\right) = 
\left(\frac{\gamma}{\bar{\gamma}}\right)^{-1/6}.
$$

In [5]:
# Step 5: Set the conformal factor variable according to the parameter BSSN_RHSs::ConformalFactor
cf = sp.sympify(0)

gammaUU, gammaDET = ixp.symm_matrix_inverter3x3(gammaDD)
gammabarUU, gammabarDET = ixp.symm_matrix_inverter3x3(gammabarDD)

if par.parval_from_str("ConformalFactor") == "phi":
    cf = sp.Rational(1,12)*sp.log(gammaDET/gammabarDET)
elif par.parval_from_str("ConformalFactor") == "chi":
    cf = (gammaDET/gammabarDET)**(-sp.Rational(1,3))
elif par.parval_from_str("ConformalFactor") == "W":
    cf = (gammaDET/gammabarDET)**(-sp.Rational(1,6))
else:
    print("Error ConformalFactor type = \""+par.parval_from_str("ConformalFactor")+"\" unknown.")
    exit(1)

Finally, we rescale tensorial quantities according to the prescription described in the [BSSN in curvilinear coordinates tutorial module](Tutorial-BSSNCurvilinear.ipynb) (also [Ruchlin *et al.*](https://arxiv.org/pdf/1712.07658.pdf)):

\begin{align}
h_{ij} &= (\bar{\gamma}_{ij} - \hat{\gamma}_{ij})/\text{ReDD[i][j]}\\
a_{ij} &= \bar{A}_{ij}/\text{ReDD[i][j]}\\
\lambda^i &= \bar{\Lambda}^i/\text{ReU[i]}\\
\mathcal{V}^i &= \beta^i/\text{ReU[i]}\\
\mathcal{B}^i &= B^i/\text{ReU[i]}\\
\end{align}

In [6]:
# Step 6: Rescale all tensorial quantities.
hDD     = ixp.zerorank2()
aDD     = ixp.zerorank2()
lambdaU = ixp.zerorank1()
vetU    = ixp.zerorank1()
betU    = ixp.zerorank1()
for i in range(DIM):
    lambdaU[i] = LambdabarU[i] / rfm.ReU[i]
    vetU[i]    =      betaU[i] / rfm.ReU[i]
    betU[i]    =         BU[i] / rfm.ReU[i]
    for j in range(DIM):
        hDD[i][j] = (gammabarDD[i][j] - rfm.ghatDD[i][j]) / rfm.ReDD[i][j]
        aDD[i][j] =                          AbarDD[i][j] / rfm.ReDD[i][j]
#print(sp.mathematica_code(hDD[0][0]))

### Code Validation: Confirm that the output from this tutorial module agrees with the module [BSSN.CartesianADMID_to_BSSNCurvilinearID](../edit/BSSN/CartesianADMID_to_BSSNCurvilinearID.py)

Here, as a code validation check, we verify agreement in the SymPy expressions for BrillLindquist initial data between
1. this tutorial and 
2. the NRPy+ BSSN module.

By default, we analyze these expressions in Spherical coordinates, though other coordinate systems may be chosen.

In [7]:
import BSSN.CartesianADMID_to_BSSNCurvilinearID as ctob
mod_cf,mod_hDD,mod_lambdaU,mod_aDD,mod_trK,mod_alpha,mod_vetU,mod_betU = \
    ctob.Convert_Cartesian_ADM_to_BSSN_curvilinear(Cartxyz, gammaCartDD, KCartDD, alphaCart, betaCartU, BCartU)

print("Consistency check between CartesianAMDID_to_BSSNCurvlinearID tutorial and NRPy+ module: ALL SHOULD BE ZERO.")

print("cf - mod_cf = " + str(cf - mod_cf))
print("trK - mod_trK = " + str(trK - mod_trK))
print("alpha - mod_alpha = " + str(alpha - mod_alpha))

for i in range(DIM):
    print("vetU["+str(i)+"] - mod_vetU["+str(i)+"] = " + str(vetU[i] - mod_vetU[i]))
    print("betU["+str(i)+"] - mod_betU["+str(i)+"] = " + str(betU[i] - mod_betU[i]))
    print("lambdaU["+str(i)+"] - mod_lambdaU["+str(i)+"] = " + str(lambdaU[i] - mod_lambdaU[i]))
    for j in range(DIM):
        print("hDD["+str(i)+"]["+str(j)+"] - mod_hDD["+str(i)+"]["+str(j)+"] = " 
              + str(hDD[i][j] - mod_hDD[i][j]))
        print("aDD["+str(i)+"]["+str(j)+"] - mod_aDD["+str(i)+"]["+str(j)+"] = " 
              + str(aDD[i][j] - mod_aDD[i][j]))

Consistency check between CartesianAMDID_to_BSSNCurvlinearID tutorial and NRPy+ module: ALL SHOULD BE ZERO.
cf - mod_cf = 0
trK - mod_trK = 0
alpha - mod_alpha = 0
vetU[0] - mod_vetU[0] = 0
betU[0] - mod_betU[0] = 0
lambdaU[0] - mod_lambdaU[0] = 0
hDD[0][0] - mod_hDD[0][0] = 0
aDD[0][0] - mod_aDD[0][0] = 0
hDD[0][1] - mod_hDD[0][1] = 0
aDD[0][1] - mod_aDD[0][1] = 0
hDD[0][2] - mod_hDD[0][2] = 0
aDD[0][2] - mod_aDD[0][2] = 0
vetU[1] - mod_vetU[1] = 0
betU[1] - mod_betU[1] = 0
lambdaU[1] - mod_lambdaU[1] = 0
hDD[1][0] - mod_hDD[1][0] = 0
aDD[1][0] - mod_aDD[1][0] = 0
hDD[1][1] - mod_hDD[1][1] = 0
aDD[1][1] - mod_aDD[1][1] = 0
hDD[1][2] - mod_hDD[1][2] = 0
aDD[1][2] - mod_aDD[1][2] = 0
vetU[2] - mod_vetU[2] = 0
betU[2] - mod_betU[2] = 0
lambdaU[2] - mod_lambdaU[2] = 0
hDD[2][0] - mod_hDD[2][0] = 0
aDD[2][0] - mod_aDD[2][0] = 0
hDD[2][1] - mod_hDD[2][1] = 0
aDD[2][1] - mod_aDD[2][1] = 0
hDD[2][2] - mod_hDD[2][2] = 0
aDD[2][2] - mod_aDD[2][2] = 0
